# SAF Taxonomy Overview

In [1]:
import core.safinfer_io as safinfer_io
import saflib.microarchitecture.TaxoRegistry as TaxoRegistry
import sys, pandas as pd
from tabulate import tabulate
print(sys.executable)

/usr/local/bin/conda/bin/python


In [2]:
#!{sys.executable} -m pip install tabulate

In [3]:
# Load taxonomic SAF microarchitecture library into SAFTools taxonomy registry
taxo_script_lib=['./microarchitecture/taxoscript/*.taxoscript.yaml']
safinfer_io.load_parse_taxo_libs(taxo_script_lib)

In [4]:
# Summary of taxonomy
print("SAF microarchitecture taxonomy")
print("")
print("- Primitive components:")
print("")
print("-- Primitive template declarations:")
print("")
primitive_template_parameters=TaxoRegistry.getPrimitiveTemplateParameterDeclarationsDict()
print(TaxoRegistry.getPrimitiveTemplateParameterDeclarationsDict())
print("")
print("-- Primitive template specializations:")
print("")
primitive_template_specializations=TaxoRegistry.getPrimitiveTemplateSpecializationsDict()
print(TaxoRegistry.getPrimitiveTemplateSpecializationsDict())
print("")
print("- Compound components:")
print("")
print("-- Compound component template declarations:")
print("")
compound_template_parameters=TaxoRegistry.getComponentTemplateParameterDeclarationsDict()
print(compound_template_parameters)
print("")
print("-- Compound component template specializations:")
print("")
compound_template_specializations=TaxoRegistry.getComponentTemplateSpecializationsDict()
print(compound_template_specializations)

SAF microarchitecture taxonomy

- Primitive components:

-- Primitive template declarations:

{'DoublePositionGenerator': {'format_src': {'idx': 0, 'values': None}, 'format_dst': {'idx': 1, 'values': None}, 'source_strategy': {'idx': 2, 'values': ['kogge_stone_prefix_sum', 'ripple_prefix_sum']}, 'reference_strategy': {'idx': 3, 'values': ['parallel_dec2_priority_encoder']}}, 'FillOptimizer': {'strategy': {'idx': 0, 'values': ['pipeline_bubble_gate', 'lut_skip']}}, 'IntersectionBidirectional': {'format_0': {'idx': 0, 'values': None}, 'format_1': {'idx': 1, 'values': None}, 'strategy': {'idx': 2, 'values': ['two_finger_merge', 'skip_ahead', 'direct_map', 'bitwise_and']}}, 'IntersectionLeaderFollower': {'format_leader': {'idx': 0, 'values': None}, 'strategy': {'idx': 1, 'values': ['passthrough']}}, 'MetadataParser': {'format': {'idx': 0, 'values': None}, 'strategy': {'idx': 1, 'values': ['sentinel', 'length_field']}, 'coordinate_arithmetic': {'idx': 2, 'values': ['no_arithmetic', 'yes_ari

In [5]:
# Redefining the generate_latex_table function to include column headers
def generate_latex_table(data, caption, label_suffix):
    """
    Generates a LaTeX table with a caption and label, including column headers.

    Parameters:
    data (list of lists): The table data, where each inner list is a row.
    caption (str): The caption for the table.
    label_suffix (str): The part of the label that comes after 'tab:'.

    Returns:
    str: A string containing the LaTeX table code.
    """
    # Generate the LaTeX table with headers
    latex_table = tabulate(data, headers="firstrow", tablefmt="latex_booktabs")

    # Complete label
    label = f"tab:{label_suffix}"

    # Format the complete LaTeX table with caption and label
    complete_latex_table = f"""
\\begin{{table}}[ht]
\\centering
{latex_table}
\\caption{{{caption}}}
\\label{{{label}}}
\\end{{table}}
"""

    return complete_latex_table

In [6]:
example_data = [["Name", "Age"], ["Alice", 24], ["Bob", 30], ["Charlie", 18]]
example_caption = "Example Table Caption"
example_label_suffix = "example_table"

latex_output = generate_latex_table(example_data, example_caption, example_label_suffix)
print(latex_output)


\begin{table}[ht]
\centering
\begin{tabular}{lr}
\toprule
 Name    &   Age \\
\midrule
 Alice   &    24 \\
 Bob     &    30 \\
 Charlie &    18 \\
\bottomrule
\end{tabular}
\caption{Example Table Caption}
\label{tab:example_table}
\end{table}



In [7]:
# Redefining the generate_specialization_table function to pass headers to LaTeX table
def generate_specialization_table(template_parameters, template_specializations, template_name):
    """
    Generates and prints a DataFrame and LaTeX table for a given template's specializations.

    Parameters:
    template_parameters (dict): Dictionary of template parameter declarations.
    template_specializations (dict): Dictionary of template specializations.
    template_name (str): The name of the template to generate the table for.

    Returns:
    str: A string containing the LaTeX table code.
    """
    
    # Check if the template exists in both dictionaries
    if template_name not in template_parameters or template_name not in template_specializations:
        return "Template name not found in provided data."

    # Define caption and label suffix at the beginning
    caption = f"Specializations of {template_name}"
    label_suffix = f"{template_name}_specializations"
    
    # Retrieve the parameters and their order
    parameters = template_parameters[template_name]
    sorted_parameters = sorted(parameters.items(), key=lambda x: x[1]['idx'])

    # Prepare the header row with parameter names
    header_row = [param_name for param_name, _ in sorted_parameters]

    # Prepare the data rows for each specialization
    data_rows = []
    for spec_id, spec_values in template_specializations[template_name].items():
        # Sort values based on the parameter order
        sorted_values = [spec_values[parameters[param_name]['idx']] for param_name, _ in sorted_parameters]
        data_rows.append(sorted_values)

    # Create DataFrame and display it
    df = pd.DataFrame(data_rows, columns=header_row)
    print(f"DataFrame for {template_name}:\n{df}\n")

    # Include the header row in the data for LaTeX table
    complete_data = [header_row] + df.to_numpy().tolist()

    # Generate the LaTeX table with headers
    latex_table = generate_latex_table(complete_data, caption, label_suffix)

    return latex_table

In [8]:
def generate_all_specialization_tables(template_parameters, template_specializations):
    """
    Generates LaTeX tables for all primitive component templates' specializations.

    Parameters:
    template_parameters (dict): Dictionary of template parameter declarations.
    template_specializations (dict): Dictionary of template specializations.

    Returns:
    dict: A dictionary containing LaTeX table codes for each template.
    """
    latex_tables = {}
    for template_name in template_parameters.keys():
        latex_table = generate_specialization_table(template_parameters, template_specializations, template_name)
        latex_tables[template_name] = latex_table
    return latex_tables

In [9]:
all_latex_tables = generate_all_specialization_tables(primitive_template_parameters, primitive_template_specializations)

# Printing out the LaTeX table for each template for demonstration
for template_name, latex_table in all_latex_tables.items():
    print(f"LaTeX Table for {template_name}:\n{latex_table}\n")

DataFrame for DoublePositionGenerator:
  format_src format_dst         source_strategy  \
0          B          B       ripple_prefix_sum   
1          B          B  kogge_stone_prefix_sum   

               reference_strategy  
0  parallel_dec2_priority_encoder  
1  parallel_dec2_priority_encoder  

DataFrame for FillOptimizer:
               strategy
0  pipeline_bubble_gate
1              lut_skip

DataFrame for IntersectionBidirectional:
  format_0 format_1          strategy
0        C        C  two_finger_merge
1        C        C        skip_ahead
2        C        C        direct_map
3        B        B       bitwise_and

DataFrame for IntersectionLeaderFollower:
  format_leader     strategy
0             C  passthrough
1             B  passthrough
2             R  passthrough

DataFrame for MetadataParser:
   format      strategy coordinate_arithmetic
0       U      sentinel         no_arithmetic
1       U      sentinel        yes_arithmetic
2       U  length_field         no_ar

In [10]:
def generate_component_table(component_parameters, component_specializations, component_name):
    """
    Generates and prints a DataFrame and LaTeX table for a given component's specializations.

    Parameters:
    component_parameters (dict): Dictionary of component parameter declarations.
    component_specializations (dict): Dictionary of component specializations.
    component_name (str): The name of the component to generate the table for.

    Returns:
    str: A string containing the LaTeX table code.
    """
    caption = f"Specializations of {component_name}"
    label_suffix = f"{component_name}_specializations"

    if component_name not in component_parameters or component_name not in component_specializations:
        return "Component name not found in provided data."

    parameters = component_parameters[component_name]
    sorted_parameters = sorted(parameters.items(), key=lambda x: x[1]['idx'])
    header_row = [param_name for param_name, _ in sorted_parameters]

    data_rows = []
    for spec_id, spec_values in component_specializations[component_name].items():
        sorted_values = [spec_values[parameters[param_name]['idx']] for param_name, _ in sorted_parameters]
        data_rows.append(sorted_values)

    df = pd.DataFrame(data_rows, columns=header_row)
    print(f"DataFrame for {component_name}:\n{df}\n")

    complete_data = [header_row] + df.to_numpy().tolist()
    latex_table = generate_latex_table(complete_data, caption, label_suffix)

    return latex_table

In [11]:
def generate_all_component_tables(component_parameters, component_specializations):
    """
    Generates LaTeX tables for all components' specializations.

    Parameters:
    component_parameters (dict): Dictionary of component parameter declarations.
    component_specializations (dict): Dictionary of component specializations.

    Returns:
    dict: A dictionary containing LaTeX table codes for each component.
    """
    latex_tables = {}
    for component_name in component_parameters.keys():
        latex_table = generate_component_table(component_parameters, component_specializations, component_name)
        latex_tables[component_name] = latex_table
    return latex_tables

In [12]:
# Generate and print LaTeX tables for all components
all_component_latex_tables = generate_all_component_tables(compound_template_parameters, compound_template_specializations)
for component_name, latex_table in all_component_latex_tables.items():
    print(f"LaTeX Table for {component_name}:\n{latex_table}\n")

DataFrame for FormatUarch:
  fibertree
0         /

DataFrame for SkippingUarch:
  format_leader format_follower        direction optimize_fills
0             C               C    bidirectional   no_opt_fills
1             B               B    bidirectional   no_opt_fills
2             C               U  leader_follower   no_opt_fills
3             C               U  leader_follower  yes_opt_fills

LaTeX Table for FormatUarch:

\begin{table}[ht]
\centering
\begin{tabular}{l}
\toprule
 fibertree   \\
\midrule
 /           \\
\bottomrule
\end{tabular}
\caption{Specializations of FormatUarch}
\label{tab:FormatUarch_specializations}
\end{table}


LaTeX Table for SkippingUarch:

\begin{table}[ht]
\centering
\begin{tabular}{llll}
\toprule
 format\_leader   & format\_follower   & direction       & optimize\_fills   \\
\midrule
 C               & C                 & bidirectional   & no\_opt\_fills     \\
 B               & B                 & bidirectional   & no\_opt\_fills     \\
 C        